In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd
import numpy as np

In [39]:
year = 2022
month = 3

In [40]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [41]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [42]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [43]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

Question 1: Standard deviation for February 2022 Yellow dataset

In [7]:
y_pred.std()

5.28140357655334

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [24]:
df_result = df['ride_id'].to_frame().copy()
df_result["pred"] = y_pred

In [35]:
df_result.to_parquet(
    "data/df_result.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

Question 2: The size of the output file

In [36]:
ls data/ -l --block-size=M

total 58M
-rw-r--r-- 1 kirillkruglikov kirillkruglikov 58M Jun 18 15:43 df_result.parquet


In [44]:
y_pred.mean()

12.758556818790902